In [66]:
import polars as pl
import pandas as pd
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm

def gen_spc_file_text(data_filename):
    spc_text = r"""series{
  file=""" + data_filename + r"""
  period=12
  format=datevalue
}
transform{
  function=auto
}
x11{ save=d11 }"""
    return spc_text
def convert_leading_zeroes_to_nan(df):
    def replace_leading_zeroes(series):
        non_zero_found = False
        new_series = []
        for value in series:
            if value != 0:
                non_zero_found = True
            if not non_zero_found and value == 0:
                new_series.append(np.nan)
            else:
                new_series.append(value)
        return pd.Series(new_series, index=series.index)
    
    return df.apply(replace_leading_zeroes, axis=0)
    
def list_txt_files(directory):
    # List all files in the specified directory
    files = os.listdir(directory)
    # Filter and return only files that end with .txt
    txt_files = [file for file in files if file.endswith('.txt')]
    return txt_files
def read_csv_and_prep_df(filename, midpoint = True):
    df = pd.read_csv(filename)
    df = convert_leading_zeroes_to_nan(df)
    if midpoint:
        for col in df.columns[2:]: df[col] = df[col].astype(float)
        df.iloc[:, 2:] = df.iloc[:, 2:].replace(0., np.nan)
        df = df.interpolate(method='linear', limit = 100, limit_area = 'inside')
    df.columns = [i.replace(' - ', '__').replace(' ', '_') for i in df.columns]
    return df
def make_x13_files_from_df(df, folder):
    os.system(f'mkdir -p {folder}')
    for var in df.columns:
        if var not in ['YEAR', 'MONTH']:
            df[['YEAR', 'MONTH', var]].dropna().to_csv(f'{folder}{var}.txt', sep=' ', index=False, header=False)
    for raw_data_file in list_txt_files(folder):
        file = open(f'{folder}spec_' + raw_data_file[:-4] + '.spc', 'w')
        file.write(gen_spc_file_text(raw_data_file))
        file.close()
def make_x13_files_from_csv(filename, folder, midpoint = True):
    df = read_csv_and_prep_df(filename, midpoint = True)
    make_x13_files_from_df(df, folder)

In [79]:
raw_data_file = 'raw_perms_series_1960_2024_vals.csv'
target_folder = 'raw_perms_midpoint_1960_2024/'
make_x13_files_from_csv(raw_data_file, target_folder)
os.system(f'cp run_seasonal_adjustment.sh {target_folder}')
os.system(f'cp x13as_html {target_folder}')
os.system(f'chmod +x {target_folder}run_seasonal_adjustment.sh')
os.system(f'chmod +x {target_folder}x13as_html')
os.system(f'./run_seasonal_adjustment.sh')

0